# Import

In [13]:
# Work with paths
import sys
sys.path.append('..')

# import implemented modules
import src.config as cfg
from src.data import make_dataset

# import other modules
from sklearn import train_test_split

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import sklearn


# Generate Dataset

In [25]:
! python3 ../src/data/make_dataset.py ../data/raw/train.csv ../data/processed/train.pkl ../data/processed/target.pkl

2022-10-15 15:18:36,119 - __main__ - INFO - making final data set from raw data
2022-10-15 15:18:36,133 - __main__ - INFO - Target saved to ../data/processed/target.pkl
2022-10-15 15:18:36,133 - __main__ - INFO - Dataset saved to ../data/processed/train.pkl


In [24]:
# I don't know how to make this work in the notebook

# make_dataset.main(  input_filepath          =cfg.RAW_DATA_PATH, 
#                     output_data_filepath    =cfg.PROCESSED_DATA_PATH, 
#                     output_target_filepath  =cfg.PROCESSED_TARGET_PATH
#                 )

Load dataset

In [40]:
dataset = pd.read_pickle(str(os.getcwd() + "/../data/processed/train.pkl"))

In [41]:
dataset

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,...,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,...,NaN,0,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,...,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,...,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,1,...,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,1,0,0,...,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,...,10.0,0,NaN,ранее употреблял,26.0,21:30:00,07:15:00,0,0,0
54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,служащие,0,1,0,...,10.0,0,NaN,употребляю в настоящее время,20.0,22:30:00,08:00:00,1,0,0


Generate tran and test selections